In [1]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

train = train.reshape(1229932, 11, 39)
test = test.reshape(451552, 11, 39)

train = train.reshape(1229932, -1)
test = test.reshape(451552, -1)

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


In [2]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

In [3]:
VAL_RATIO = 0

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1229932, 429)
Size of validation set: (0, 429)


In [4]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

45

In [6]:
import torch
import torch.nn as nn

class Classifier1(nn.Module):
    def __init__(self):
        super(Classifier1, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(429, 2048), # 1
            nn.LeakyReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048, 2048), # 2
            nn.LeakyReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048, 2048), # 2
            nn.LeakyReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048,1024), # 3
            nn.LeakyReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 512), # 4
            nn.LeakyReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 256), # 5
            nn.LeakyReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, 39)
        )

    def forward(self, x):
        x = self.net(x)
        return x

In [7]:
class Classifier2(nn.Module):
    def __init__(self):
        super(Classifier2, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(429, 2048), # 1
            nn.LeakyReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048, 2048), # 2
            nn.LeakyReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048,1024), # 3
            nn.LeakyReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 512), # 4
            nn.LeakyReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 128), # 5
            nn.LeakyReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, 39)
        )

    def forward(self, x):
        x = self.net(x)
        return x

In [8]:
class Classifier3(nn.Module):
    def __init__(self):
        super(Classifier3, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(429, 2048), # 1
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048, 2048), # 2
            nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048,1024), # 3
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024), # 4
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 512), # 5
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 128), # 6
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, 39)
        )

    def forward(self, x):
        x = self.net(x)
        return x

In [9]:
#check device
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [11]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# the path where checkpoint saved
model1_path = './modeldnn.ckpt' # Train Acc: 0.806587 Loss: 0.577284

model2_path = './model2dnn.ckpt' # Train Acc: 0.777302 Loss: 0.708116

model3_path = './model3dnn.ckpt' # Train Acc: 0.795503 Loss: 0.632341

DEVICE: cuda


In [12]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model1 = Classifier1().to(device)
model1.load_state_dict(torch.load(model1_path)['model_state_dict'])

model2 = Classifier2().to(device)
model2.load_state_dict(torch.load(model2_path)['model_state_dict'])

model3 = Classifier3().to(device)
model3.load_state_dict(torch.load(model3_path)['model_state_dict'])

<All keys matched successfully>

# Predict

In [13]:
predict = []
model1.eval() # set the model to evaluation mode
model2.eval() # set the model to evaluation mode
model3.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs1 = model1(inputs)
        outputs2 = model2(inputs)
        outputs3 = model3(inputs)
        outputs = (outputs1 + outputs2 + outputs3) / 3
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

# Post-process

In [14]:
count = 0
for i in range(1, len(predict)-1):
    previous_ = predict[i-1]
    next_ = predict[i+1]
    current_ = predict[i]
    if (previous_ != current_) and (next_ != current_) and (previous_ == next_):
        print('idx',i,'correct', current_, 'to', previous_)
        predict[i] = previous_
        count +=1

print('total number of correction %d, correction percent %.2f'% (count, count/len(predict)))

idx 558 correct 11 to 3
idx 597 correct 3 to 11
idx 965 correct 11 to 3
idx 1419 correct 4 to 13
idx 1451 correct 17 to 1
idx 1619 correct 2 to 1
idx 1688 correct 19 to 20
idx 1831 correct 4 to 1
idx 1859 correct 6 to 4
idx 2088 correct 7 to 9
idx 2097 correct 7 to 9
idx 2105 correct 21 to 36
idx 2111 correct 36 to 33
idx 2363 correct 4 to 12
idx 2367 correct 4 to 12
idx 2793 correct 5 to 0
idx 2825 correct 25 to 38
idx 3272 correct 13 to 7
idx 3434 correct 3 to 2
idx 3473 correct 2 to 1
idx 3881 correct 38 to 32
idx 4044 correct 38 to 35
idx 4093 correct 3 to 9
idx 4161 correct 9 to 7
idx 4265 correct 21 to 35
idx 4371 correct 25 to 38
idx 4514 correct 12 to 13
idx 4660 correct 35 to 36
idx 4710 correct 0 to 8
idx 4810 correct 32 to 38
idx 4965 correct 11 to 3
idx 4968 correct 18 to 19
idx 5145 correct 17 to 14
idx 5245 correct 17 to 1
idx 5295 correct 4 to 1
idx 5557 correct 4 to 1
idx 5559 correct 4 to 1
idx 5678 correct 3 to 2
idx 6098 correct 18 to 19
idx 6326 correct 1 to 17
idx 

In [15]:
with open('prediction_ensemble.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

# Reference
The code is modified from TA's sample code